In [ ]:
### Image acquisition: 210508
### 1. Import packages
### 2. Connect Motion Controller
### 3. Connect Lock-in Amplifier
### 4. Check signal
### 5. Set initial position
### 6. Set final position
### 7. Acquire an image
### 8. Save the image

In [ ]:
### 1. Import packages
import io # check blas in skimage
import numpy as np
import matplotlib.pyplot as plt
import serial
import time
from PIL import Image
import progressbar
from scipy import ndimage

In [ ]:
### 2. Connect motion controller
motion_controller = serial.Serial()
motion_controller.baudrate = 19200
motion_controller.port = 'COM6'
motion_controller.rts=True
motion_controller.dtr=True
motion_controller.open()

print("Motion Controller connected to: " + motion_controller.portstr)

In [ ]:
### 3. Connect Lock-in Amplifier
lock_in_amplifier = serial.Serial()
lock_in_amplifier.baudrate = 19200
lock_in_amplifier.timeout=0.2
lock_in_amplifier.port = 'COM9'
lock_in_amplifier.rts=True
lock_in_amplifier.dtr=True
lock_in_amplifier.open()

print("Lock-in amplifier connected to: " + lock_in_amplifier.portstr)

In [ ]:
### 4. Check signal
cmd_lock_in_amplifier='Q\r' # read Voltage (V)
lock_in_amplifier.write(cmd_lock_in_amplifier.encode('ascii'))

value=str(lock_in_amplifier.read(lock_in_amplifier.inWaiting()).decode('utf-8'))[0:-1] #remove\r
print(float(np.array(value.rsplit('\r',2))))

In [ ]:
### 5. Set initial position
pos_X= -10.0 #mm  #Axis 2
pos_Y= -10.0 #mm #Axis 1

time.sleep(2) 
#set low speed
cmd_motion_controller='1VA5.0\r\n'
motion_controller.write(cmd_motion_controller.encode('ascii'))
cmd_motion_controller='2VA5.0\r\n'
motion_controller.write(cmd_motion_controller.encode('ascii'))

cmd_motion_controller='1PA'+str(round(pos_Y, 4))+'\r\n'
motion_controller.write(cmd_motion_controller.encode('ascii'))
cmd_motion_controller='2PA'+str(round(pos_X, 4))+'\r\n'
motion_controller.write(cmd_motion_controller.encode('ascii'))
time.sleep(1) 

In [ ]:
### 6. Set final position
final_pos_X = 0.0 #mm  #Axis 2
final_pos_Y = 0.0 #mm #Axis 1 

step_size=1 #mm  #never bigger thn 1 mm to avoid motor error

size_x= int(np.absolute(final_pos_X - pos_X)/step_size) #
size_y= int(np.absolute(final_pos_Y - pos_Y)/step_size)  #
print(size_x)
print(size_y)

wait_time=0.3 #0.3 ##wait time over each position 0.4
path='Documents/Data/TeraHertz/211113/'
sample='211113_noSample_ph_10mm_Overview_1_wt0.3s_300msIntTime'

img=np.zeros(shape=[size_y, size_x], dtype=float)


In [ ]:
### 7. Acquire an image
bar = progressbar.ProgressBar().start()
pos_XY=[]
#set low speed for axis 1 and 2 (y and x)
cmd_motion_controller='1VA20.0\r\n' #20.0 if step=1 mm for less 30.0
motion_controller.write(cmd_motion_controller.encode('ascii'))
cmd_motion_controller='2VA15.0\r\n'
motion_controller.write(cmd_motion_controller.encode('ascii'))
##Right to Left, Bottom Up
mylist=[]
for y in range(0,size_y):
    #plt.imshow(img)
    cmd_motion_controller='1PA'+str(round(pos_Y+(y*step_size), 4))+'\r\n'
    motion_controller.write(cmd_motion_controller.encode('ascii'))
    #print(str(round(pos_Y+(y*step_size), 4)))
    part= Image.fromarray(img)
    part.save(path+sample+'_start_'+str(pos_X)+'_'+str(pos_Y)+'_'+str(size_x)+'_pix-'+str(size_x)+'_pix'+'_sampling_'+str(step_size)+'_mm-PARTIAL.tiff')
    for x in range(0,size_x):
        cmd_motion_controller='2PA'+str(round(pos_X+(x*step_size), 4))+'\r\n'
        motion_controller.write(cmd_motion_controller.encode('ascii'))
        
        ### TO DO: get real position
        # time.sleep(wait_time)
        # cmd_motion_controller='1PA?\r\n'
        # motion_controller.write(cmd_motion_controller.encode('ascii'))
        # val_y=motion_controller.read(motion_controller.inWaiting()).decode('utf-8') #remove\r
        # #print(val_y)
        # time.sleep(wait_time)
        # cmd_motion_controller='2PA?\r\n'
        # motion_controller.write(cmd_motion_controller.encode('ascii'))
        # val_x=motion_controller.read(motion_controller.inWaiting()).decode('utf-8')#remove\r
        # #print(val_x)
        # pos_XY.append([val_x, val_y])
        
        time.sleep(wait_time) #allow signal to stabilise?
        ###READING SIGNAL
        cmd_lock_in_amplifier='Q \r\n' # print Amplitude -> see table
        lock_in_amplifier.write(cmd_lock_in_amplifier.encode('ascii'))
        value=str(lock_in_amplifier.read(lock_in_amplifier.inWaiting()).decode('utf-8'))[0:-1] #remove\r #remove\r  #need wait_time=0.5 ?? free buffer?
        img[y,x]=value
        mylist.append(value)
        bar.update(100*(x+1)*(y+1)/(size_x*size_y))
    

In [ ]:
### 8. Save the image
rotated=ndimage.rotate(img, 180)
im = Image.fromarray(rotated) 
im.save(path+sample+'_start_'+str(pos_X)+'_'+str(pos_Y)+'_'+str(size_x)+'_pix-'+str(size_x)+'_pix'+'_sampling_'+str(step_size)+'_mm.tiff')